<a href="https://colab.research.google.com/github/RyanUdeM/De-Bruijn-Graph/blob/master/19birds_gff2csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2: Bird introns
### *Xiao Ju & Samuel Briand*

In [0]:
import pandas as pd

### import gffpandas in Colab (from local drive)
Importing gffpandas files in Google Colab 

https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab

Github of gffpandas

https://github.com/foerstner-lab/gffpandas



In [2]:
from google.colab import files
src = list(files.upload().values())[0]
open('gffpandas.py','wb').write(src)
import gffpandas as gffpd


IndexError: ignored

## 1. import gff files in Colab (from local drive), transform into Bird_CDS.csv

https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92

In [0]:
#chang the input file name and the output file name for each bird

gffFile = "Pigeon(Columba_livia).gff"  #input gff file name
csvFile = "Pigeon_CDS.csv"  #output csv file name



from google.colab import files
uploaded = files.upload()

Saving Pigeon(Columba_livia).gff to Pigeon(Columba_livia).gff


In [0]:

bird = gffpd.read_gff3(gffFile)
bird.df.head()


,seq_id,source,type,start,end,score,strand,phase,attributes
0,C16012181,GeneWise,mRNA,2,118,53.33,-,.,ID=Cli_R000001;Source=ENSTGUT00000019035;Funct...
1,C16012181,GeneWise,CDS,2,118,.,-,0,Parent=Cli_R000001;
2,C16062437,GeneWise,mRNA,3,125,55.56,-,.,ID=Cli_R000002;Source=ENSTGUT00000019035;Funct...
3,C16062437,GeneWise,CDS,3,125,.,-,0,Parent=Cli_R000002;
4,C16070423,GeneWise,mRNA,3,125,62.50,+,.,ID=Cli_R000003;Source=ENSTGUT00000015737;Funct...


#### note that in the Alligator gff file, there are different features, but in the birds gff files, we only have CDS feature:
CDS = exon - tss -tts 

CDS1-CDS2 =intron

#### so we can keep only CDS features for this study, than calculate introns from CDS for all the chosen species 

In [0]:
# keep only CDS lines
bird_cds = bird.filter_feature_of_type(['CDS']).to_gff3('bird_cds.gff')


In [0]:
bird_cds = gffpd.read_gff3('bird_cds.gff')
bird_cds.df.head()


,seq_id,source,type,start,end,score,strand,phase,attributes
0,C16012181,GeneWise,CDS,2,118,.,-,0,Parent=Cli_R000001;
1,C16062437,GeneWise,CDS,3,125,.,-,0,Parent=Cli_R000002;
2,C16070423,GeneWise,CDS,3,125,.,+,0,Parent=Cli_R000003;
3,C16084155,GeneWise,CDS,3,131,.,+,0,Parent=Cli_R000004;
4,C16088147,GeneWise,CDS,1,129,.,+,0,Parent=Cli_R000005;


In [0]:
bird_cds.df['attributes']= bird_cds.df.attributes.str.split('=').str[1].str.split(';').str[0]

In [0]:
bird_cds.df.head()

,seq_id,source,type,start,end,score,strand,phase,attributes
0,C16012181,GeneWise,CDS,2,118,.,-,0,Cli_R000001
1,C16062437,GeneWise,CDS,3,125,.,-,0,Cli_R000002
2,C16070423,GeneWise,CDS,3,125,.,+,0,Cli_R000003
3,C16084155,GeneWise,CDS,3,131,.,+,0,Cli_R000004
4,C16088147,GeneWise,CDS,1,129,.,+,0,Cli_R000005


In [0]:
bird_cds.df.to_csv(csvFile,index=False)

In [0]:
from google.colab import files
files.download(csvFile)

## 2. Create ortholog cheking matrice

### import ''name_abbreviation.txt''

In [0]:
from google.colab import files
uploaded = files.upload()

Saving name_abbreviation.txt to name_abbreviation.txt


In [0]:
abbreviation = pd.read_table('name_abbreviation.txt')
#this abbreviation include 48 birds and 4 other species (human, alligator, sea turle, lizard)
names = list(abbreviation.Short_name)


### import ''48birds_ortholog.list'' 

In [0]:
from google.colab import files
uploaded = files.upload()

Saving 48birds_ortholog.list to 48birds_ortholog (2).list


In [0]:
names.insert(0, "GeneName")
names.insert(1, "orthologID")
names.append("null")

In [0]:
ortholog = pd.read_table('48birds_ortholog.list',names = names)



In [0]:
ortholog= ortholog.drop(columns=['null'])
ortholog.head()

,GeneName,orthologID,GALGA,ACACH,ALLIG,ANAPL,ANOCA,APAVI,APTFO,BALRE,...,PHORU,PICPU,PODCR,PTEGU,PYGAD,STRCA,TAEGU,TAUER,TINGU,TYTAL
0,AAK1,5,ENSGALT00000000026_GALGA,Ach_R005076_ACACH,All_R008761_ALLIG,Apl_R007351_ANAPL,ENSACAT00000009734_ANOCA,-,Afo_R001347_APTFO,Bre_R010288_BALRE,...,Pru_R000751_PHORU,Ppu_R013258_PICPU,Pcr_R001844_PODCR,Pgu_R012240_PTEGU,Pad_R004409_PYGAD,Sca_R000199_STRCA,TGUT00000004004_TAEGU,Ter_R000935_TAUER,Tma_R003208_TINGU,Tal_R003333_TYTAL
1,SPR,12,ENSGALT00000000092_GALGA,Ach_R014163_ACACH,All_R017284_ALLIG,Apl_R007665_ANAPL,ENSACAT00000000950_ANOCA,Avi_R001982_APAVI,Afo_R006060_APTFO,Bre_R008084_BALRE,...,Pru_R009841_PHORU,Ppu_R014042_PICPU,Pcr_R005412_PODCR,Pgu_R012251_PTEGU,Pad_R006801_PYGAD,-,-,Ter_R008470_TAUER,Tma_R000919_TINGU,Tal_R010334_TYTAL
2,GFPT1,15,ENSGALT00000000115_GALGA,Ach_R010491_ACACH,All_R002153_ALLIG,Apl_R007347_ANAPL,-,Avi_R002618_APAVI,Afo_R001351_APTFO,Bre_R002202_BALRE,...,Pru_R001747_PHORU,Ppu_R000241_PICPU,Pcr_R013053_PODCR,Pgu_R009473_PTEGU,Pad_R004405_PYGAD,Sca_R000195_STRCA,TGUT00000004075_TAEGU,Ter_R006417_TAUER,Tma_R003204_TINGU,Tal_R004080_TYTAL
3,NAT10,16,ENSGALT00000000127_GALGA,Ach_R013432_ACACH,All_R005195_ALLIG,Apl_R000227_ANAPL,ENSACAT00000013625_ANOCA,Avi_R005539_APAVI,Afo_R006666_APTFO,Bre_R013256_BALRE,...,Pru_R011083_PHORU,Ppu_R015059_PICPU,Pcr_R005135_PODCR,Pgu_R004263_PTEGU,Pad_R003793_PYGAD,Sca_R005657_STRCA,ENSTGUT00000005902_TAEGU,Ter_R013163_TAUER,-,Tal_R002096_TYTAL
4,LRIF1,20,ENSGALT00000000145_GALGA,-,All_R008095_ALLIG,Apl_R003981_ANAPL,-,Avi_R009268_APAVI,Afo_R005416_APTFO,Bre_R004213_BALRE,...,Pru_R001157_PHORU,Ppu_R001132_PICPU,Pcr_R007362_PODCR,Pgu_R010685_PTEGU,Pad_R011369_PYGAD,Sca_R002995_STRCA,TGUT00000001457_TAEGU,Ter_R003469_TAUER,Tma_R012612_TINGU,Tal_R003603_TYTAL


In [0]:
ortholog.loc[0, "ACACH"]

'Ach_R005076_ACACH'

In [0]:
for col in ortholog.columns:

    for row in ortholog[col]:
        ortholog[col] = ortholog[col].map(lambda x: str(x).split("_"+col,1)[0])

In [0]:
ortholog.head()

,GeneName,orthologID,GALGA,ACACH,ALLIG,ANAPL,ANOCA,APAVI,APTFO,BALRE,...,PHORU,PICPU,PODCR,PTEGU,PYGAD,STRCA,TAEGU,TAUER,TINGU,TYTAL
0,AAK1,5,ENSGALT00000000026,Ach_R005076,All_R008761,Apl_R007351,ENSACAT00000009734,-,Afo_R001347,Bre_R010288,...,Pru_R000751,Ppu_R013258,Pcr_R001844,Pgu_R012240,Pad_R004409,Sca_R000199,TGUT00000004004,Ter_R000935,Tma_R003208,Tal_R003333
1,SPR,12,ENSGALT00000000092,Ach_R014163,All_R017284,Apl_R007665,ENSACAT00000000950,Avi_R001982,Afo_R006060,Bre_R008084,...,Pru_R009841,Ppu_R014042,Pcr_R005412,Pgu_R012251,Pad_R006801,-,-,Ter_R008470,Tma_R000919,Tal_R010334
2,GFPT1,15,ENSGALT00000000115,Ach_R010491,All_R002153,Apl_R007347,-,Avi_R002618,Afo_R001351,Bre_R002202,...,Pru_R001747,Ppu_R000241,Pcr_R013053,Pgu_R009473,Pad_R004405,Sca_R000195,TGUT00000004075,Ter_R006417,Tma_R003204,Tal_R004080
3,NAT10,16,ENSGALT00000000127,Ach_R013432,All_R005195,Apl_R000227,ENSACAT00000013625,Avi_R005539,Afo_R006666,Bre_R013256,...,Pru_R011083,Ppu_R015059,Pcr_R005135,Pgu_R004263,Pad_R003793,Sca_R005657,ENSTGUT00000005902,Ter_R013163,-,Tal_R002096
4,LRIF1,20,ENSGALT00000000145,-,All_R008095,Apl_R003981,-,Avi_R009268,Afo_R005416,Bre_R004213,...,Pru_R001157,Ppu_R001132,Pcr_R007362,Pgu_R010685,Pad_R011369,Sca_R002995,TGUT00000001457,Ter_R003469,Tma_R012612,Tal_R003603


In [0]:
ortholog.to_csv("ortholog.csv",index=False)

In [0]:
from google.colab import files
files.download("ortholog.csv")